## 1. BBDD_Final

### Objetivo



### Descripción General de notebook

    1. Carga de datos base:
        - Movimientos históricos
        - Demografía por barrio
        - Geografía por estación Bicimad
        
    2. Concatenación de datasets
    3. Exportar DataFrame a CSV

In [1]:
# %run "/Users/miguelflores/Desktop/TFM/Codigo/1. Librerias Mongo/MongoDB_Connections.ipynb"
# %run "/Users/miguelflores/Desktop/TFM/Codigo/1. Librerias Mongo/MongoDB_Funciones_Consultas.ipynb"

%run "../1. Librerias Mongo/MongoDB_Connections.ipynb"
%run "../1. Librerias Mongo/MongoDB_Funciones_Consultas.ipynb"


# from ipynb.fs.full.MongoDB_Connections import _connect_mongo

# from ipynb.fs.full.MongoDB_Funciones_Consultas import _resumen_mensual_por_anio
# from ipynb.fs.full.MongoDB_Funciones_Consultas import _data_anio
# from ipynb.fs.full.MongoDB_Funciones_Consultas import _data_anio_mes
# from ipynb.fs.full.MongoDB_Funciones_Consultas import _data_anio_por_estacion
# from ipynb.fs.full.MongoDB_Funciones_Consultas import _data_anio_mes_por_estacion

# from ipynb.fs.full.MongoDB_Funciones_Consultas import _consulta_meteo_full
# from ipynb.fs.full.MongoDB_Funciones_Consultas import _consulta_meteo_por_anio
# from ipynb.fs.full.MongoDB_Funciones_Consultas import _consulta_meteo_por_anio_mes

# from ipynb.fs.full.MongoDB_Funciones_Consultas import _consulta_stations
# from ipynb.fs.full.MongoDB_Funciones_Consultas import _consulta_stations_EstacionesMeteo

import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# pd.options.display.float_format = '{:,}'.format

## Conexión a Mongo

In [2]:
db_Tracks = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 'Tracks_Demanda')
db_Meteo = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 'Meteo_NivelHora')
db_Stations = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 'Station_Master')
db_Stations_Meteo = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 'Station_EstacionMeteo')

Conexion OK
Collection(Database(MongoClient(host=['ac-x1d17w2-shard-00-02.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-00.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-01.15npsxw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-xyv6ql-shard-0', tls=True, serverselectiontimeoutms=4000, tlscafile='C:\\ProgramData\\Anaconda3\\lib\\site-packages\\certifi\\cacert.pem'), 'BiciMAD'), 'Tracks_Demanda')
Conexion OK
Collection(Database(MongoClient(host=['ac-x1d17w2-shard-00-01.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-02.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-00.15npsxw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-xyv6ql-shard-0', tls=True, serverselectiontimeoutms=4000, tlscafile='C:\\ProgramData\\Anaconda3\\lib\\site-packages\\certifi\\cacert.pem'), 'BiciMAD'), 'Meteo_NivelHora')
Conexion OK
Collection(Database(MongoClient(host=['ac-x1d17w2-shard

In [3]:
#data_Tracks.to_csv('/Users/miguelflores/Desktop/TFM/DataFullTracks_MongoDB.csv')

In [4]:
import time
import pandas as pd
import numpy as np

t_ini = time.time()
#data_Tracks = pd.read_csv('/Users/miguelflores/Desktop/TFM/DataFullTracks_MongoDB.csv')
#data_Tracks = pd.read_csv('C:/Users/fabio/GitHub/Python Notebooks/Data/DataTracks_MongoDB_2018-2020-Ene-Feb.csv', low_memory=False)
# data_Tracks = pd.read_csv('../../Data/DataTracks_MongoDB_2018-2020-Ene-Feb.csv', parse_dates=['UNPLUG_TIME_date'])
data_Tracks = pd.read_csv('../../Data/DataTracks_MongoDB_2017-2020-Ene-Feb.csv', parse_dates=['UNPLUG_TIME_date'])
t_end = time.time()
print (t_end - t_ini)

8.674495935440063


In [5]:
estaciones = data_Tracks.idunplug_station.unique()
estaciones = pd.DataFrame(estaciones)
estaciones = estaciones.rename(columns={0:'ESTACION'})
estaciones

,ESTACION
0,10
1,102
2,108
3,109
4,110
...,...
213,206
214,204
215,197
216,209


In [6]:
data_Tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11276120 entries, 0 to 11276119
Data columns (total 17 columns):
 #   Column            Dtype         
---  ------            -----         
 0   ANIO              int64         
 1   MES               int64         
 2   DIA               int64         
 3   HORA              int64         
 4   DIA_SEMANA        int64         
 5   AM_PM             object        
 6   TEMPORADA         object        
 7   TEMPORADA_NUM     int64         
 8   UNPLUG_TIME_date  datetime64[ns]
 9   travel_time       int64         
 10  idunplug_station  int64         
 11  user_type         int64         
 12  ageRange          int64         
 13  zip_code          object        
 14  Es_Festivo        int64         
 15  Es_FinSemana      int64         
 16  DEMANDA           int64         
dtypes: datetime64[ns](1), int64(13), object(3)
memory usage: 1.4+ GB


In [7]:
data_Tracks.shape

(11276120, 17)

In [8]:
data_Tracks

,ANIO,MES,DIA,HORA,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,UNPLUG_TIME_date,travel_time,idunplug_station,user_type,ageRange,zip_code,Es_Festivo,Es_FinSemana,DEMANDA
0,2017,3,31,23,6,PM,INVIERNO,1,2017-03-31 23:00:00,1046,10,1,3,28053,0,0,1
1,2017,3,31,23,6,PM,INVIERNO,1,2017-03-31 23:00:00,1377,10,1,3,28007,0,0,1
2,2017,3,31,23,6,PM,INVIERNO,1,2017-03-31 23:00:00,497,102,1,4,28007,0,0,1
3,2017,3,31,23,6,PM,INVIERNO,1,2017-03-31 23:00:00,639,108,1,2,28009,0,0,1
4,2017,3,31,23,6,PM,INVIERNO,1,2017-03-31 23:00:00,761,109,1,3,28029,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11276115,2020,2,9,9,1,AM,INVIERNO,1,2020-02-09 09:00:00,306,84,1,4,28045.0,0,1,1
11276116,2020,2,9,9,1,AM,INVIERNO,1,2020-02-09 09:00:00,253,89,1,4,28014.0,0,1,1
11276117,2020,2,9,9,1,AM,INVIERNO,1,2020-02-09 09:00:00,1450,90,1,5,28300.0,0,1,1
11276118,2020,2,9,9,1,AM,INVIERNO,1,2020-02-09 09:00:00,519,90,1,5,28028.0,0,1,1


### Quitar User_type = 3 (Trabajador BICIMAD)

In [9]:
user_type_dif_3 = data_Tracks['user_type'] != 3
data_Tracks = data_Tracks[user_type_dif_3]
data_Tracks.shape

(10440992, 17)

In [10]:
data_Tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10440992 entries, 0 to 11276119
Data columns (total 17 columns):
 #   Column            Dtype         
---  ------            -----         
 0   ANIO              int64         
 1   MES               int64         
 2   DIA               int64         
 3   HORA              int64         
 4   DIA_SEMANA        int64         
 5   AM_PM             object        
 6   TEMPORADA         object        
 7   TEMPORADA_NUM     int64         
 8   UNPLUG_TIME_date  datetime64[ns]
 9   travel_time       int64         
 10  idunplug_station  int64         
 11  user_type         int64         
 12  ageRange          int64         
 13  zip_code          object        
 14  Es_Festivo        int64         
 15  Es_FinSemana      int64         
 16  DEMANDA           int64         
dtypes: datetime64[ns](1), int64(13), object(3)
memory usage: 1.4+ GB


## travel_time > 90 seg

In [11]:
user_travel = data_Tracks['travel_time'] > 90
data_Tracks = data_Tracks[user_travel]
data_Tracks.shape

(10431882, 17)

In [12]:
data_Tracks.rename(columns = {'idunplug_station':'ESTACION'}, inplace = True)
data_Tracks

,ANIO,MES,DIA,HORA,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,UNPLUG_TIME_date,travel_time,ESTACION,user_type,ageRange,zip_code,Es_Festivo,Es_FinSemana,DEMANDA
0,2017,3,31,23,6,PM,INVIERNO,1,2017-03-31 23:00:00,1046,10,1,3,28053,0,0,1
1,2017,3,31,23,6,PM,INVIERNO,1,2017-03-31 23:00:00,1377,10,1,3,28007,0,0,1
2,2017,3,31,23,6,PM,INVIERNO,1,2017-03-31 23:00:00,497,102,1,4,28007,0,0,1
3,2017,3,31,23,6,PM,INVIERNO,1,2017-03-31 23:00:00,639,108,1,2,28009,0,0,1
4,2017,3,31,23,6,PM,INVIERNO,1,2017-03-31 23:00:00,761,109,1,3,28029,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11276115,2020,2,9,9,1,AM,INVIERNO,1,2020-02-09 09:00:00,306,84,1,4,28045.0,0,1,1
11276116,2020,2,9,9,1,AM,INVIERNO,1,2020-02-09 09:00:00,253,89,1,4,28014.0,0,1,1
11276117,2020,2,9,9,1,AM,INVIERNO,1,2020-02-09 09:00:00,1450,90,1,5,28300.0,0,1,1
11276118,2020,2,9,9,1,AM,INVIERNO,1,2020-02-09 09:00:00,519,90,1,5,28028.0,0,1,1


## Agrdupación BBDD

In [13]:
data_final = data_Tracks[['ANIO','MES','DIA','HORA','ESTACION','DIA_SEMANA','AM_PM','TEMPORADA','TEMPORADA_NUM', 'Es_Festivo', 'Es_FinSemana','DEMANDA']]
data_final = data_final.groupby(['ANIO','MES','DIA','HORA','ESTACION','DIA_SEMANA','AM_PM','TEMPORADA','TEMPORADA_NUM', 'Es_Festivo', 'Es_FinSemana']).agg(DEMANDA=('DEMANDA', 'sum'))
data_final = data_final.reset_index()
data_final

,ANIO,MES,DIA,HORA,ESTACION,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,Es_Festivo,Es_FinSemana,DEMANDA
0,2017,3,31,23,1,6,PM,INVIERNO,1,0,0,7
1,2017,3,31,23,2,6,PM,INVIERNO,1,0,0,3
2,2017,3,31,23,3,6,PM,INVIERNO,1,0,0,1
3,2017,3,31,23,4,6,PM,INVIERNO,1,0,0,1
4,2017,3,31,23,5,6,PM,INVIERNO,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
3033210,2020,2,29,23,214,7,PM,INVIERNO,1,0,1,2
3033211,2020,2,29,23,215,7,PM,INVIERNO,1,0,1,1
3033212,2020,2,29,23,216,7,PM,INVIERNO,1,0,1,1
3033213,2020,2,29,23,218,7,PM,INVIERNO,1,0,1,1


### Creacion de campo FECHA

In [14]:
data_final['FECHA'] =pd.to_datetime({'year': data_final['ANIO'],
                                          'month': data_final["MES"]
                                          ,'day':  data_final['DIA']
                                          ,'hour':data_final['HORA']},
                                          format='%d-%m-%Y', errors='coerce')
data_final

,ANIO,MES,DIA,HORA,ESTACION,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,Es_Festivo,Es_FinSemana,DEMANDA,FECHA
0,2017,3,31,23,1,6,PM,INVIERNO,1,0,0,7,2017-03-31 23:00:00
1,2017,3,31,23,2,6,PM,INVIERNO,1,0,0,3,2017-03-31 23:00:00
2,2017,3,31,23,3,6,PM,INVIERNO,1,0,0,1,2017-03-31 23:00:00
3,2017,3,31,23,4,6,PM,INVIERNO,1,0,0,1,2017-03-31 23:00:00
4,2017,3,31,23,5,6,PM,INVIERNO,1,0,0,2,2017-03-31 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033210,2020,2,29,23,214,7,PM,INVIERNO,1,0,1,2,2020-02-29 23:00:00
3033211,2020,2,29,23,215,7,PM,INVIERNO,1,0,1,1,2020-02-29 23:00:00
3033212,2020,2,29,23,216,7,PM,INVIERNO,1,0,1,1,2020-02-29 23:00:00
3033213,2020,2,29,23,218,7,PM,INVIERNO,1,0,1,1,2020-02-29 23:00:00


In [15]:
data_estaciones = data_final[['ESTACION','FECHA']]
data_estaciones = data_estaciones.groupby('ESTACION').agg(min=('FECHA', 'min'), max=('FECHA', 'max'))
data_estaciones = data_estaciones.reset_index()
data_estaciones

,ESTACION,min,max
0,1,2017-03-31 23:00:00,2020-02-29 22:00:00
1,2,2017-03-31 23:00:00,2020-02-29 23:00:00
2,3,2017-03-31 23:00:00,2020-02-29 23:00:00
3,4,2017-03-31 23:00:00,2020-02-29 21:00:00
4,5,2017-03-31 23:00:00,2020-02-29 23:00:00
...,...,...,...
213,216,2019-09-12 11:00:00,2020-02-29 23:00:00
214,217,2019-09-11 08:00:00,2020-02-29 22:00:00
215,218,2019-07-10 14:00:00,2020-02-29 23:00:00
216,219,2019-08-07 11:00:00,2020-02-29 23:00:00


### FILTRO 1: BUSQUEDA DE ESTACIONES CON MOVIMIENTOS MENORES A 1 AÑO

In [16]:
data_estaciones

,ESTACION,min,max
0,1,2017-03-31 23:00:00,2020-02-29 22:00:00
1,2,2017-03-31 23:00:00,2020-02-29 23:00:00
2,3,2017-03-31 23:00:00,2020-02-29 23:00:00
3,4,2017-03-31 23:00:00,2020-02-29 21:00:00
4,5,2017-03-31 23:00:00,2020-02-29 23:00:00
...,...,...,...
213,216,2019-09-12 11:00:00,2020-02-29 23:00:00
214,217,2019-09-11 08:00:00,2020-02-29 22:00:00
215,218,2019-07-10 14:00:00,2020-02-29 23:00:00
216,219,2019-08-07 11:00:00,2020-02-29 23:00:00


In [17]:
data_estaciones['min'] = pd.to_datetime(data_estaciones['min'])
data_estaciones['max'] = pd.to_datetime(data_estaciones['max'])

In [18]:
data_estaciones['Diferencia'] = (data_estaciones["max"] - data_estaciones["min"]).dt.days
data_estaciones

,ESTACION,min,max,Diferencia
0,1,2017-03-31 23:00:00,2020-02-29 22:00:00,1064
1,2,2017-03-31 23:00:00,2020-02-29 23:00:00,1065
2,3,2017-03-31 23:00:00,2020-02-29 23:00:00,1065
3,4,2017-03-31 23:00:00,2020-02-29 21:00:00,1064
4,5,2017-03-31 23:00:00,2020-02-29 23:00:00,1065
...,...,...,...,...
213,216,2019-09-12 11:00:00,2020-02-29 23:00:00,170
214,217,2019-09-11 08:00:00,2020-02-29 22:00:00,171
215,218,2019-07-10 14:00:00,2020-02-29 23:00:00,234
216,219,2019-08-07 11:00:00,2020-02-29 23:00:00,206


In [19]:
data_estaciones_incompleta = data_estaciones[data_estaciones['Diferencia'] < 789]
print(data_estaciones_incompleta.shape)
data_estaciones_incompleta.sort_values('Diferencia', ascending=False)

(47, 4)


,ESTACION,min,max,Diferencia
22,23,2017-03-31 23:00:00,2018-04-26 05:00:00,390
23,24,2017-03-31 23:00:00,2018-04-25 08:00:00,389
208,211,2019-05-23 09:00:00,2020-02-29 22:00:00,282
187,190,2019-05-23 16:00:00,2020-02-29 23:00:00,282
181,184,2019-06-07 11:00:00,2020-02-29 20:00:00,267
186,189,2019-06-07 11:00:00,2020-02-29 23:00:00,267
197,200,2019-06-10 12:00:00,2020-02-29 21:00:00,264
198,201,2019-06-11 13:00:00,2020-02-29 21:00:00,263
182,185,2019-06-11 08:00:00,2020-02-29 22:00:00,263
210,213,2019-06-13 13:00:00,2020-02-29 21:00:00,261


In [20]:
data_estaciones_incompleta[data_estaciones_incompleta['Diferencia']<365].shape

(45, 4)

In [21]:
# FILTRO: SE ELIMINAN ESTACIONES CON MENOS DE 365 DIAS DE MOVIMIENTOS
# ESTACIONES CORRECTAS: 171

df_estaciones_365 = data_estaciones[data_estaciones['Diferencia']>=365]['ESTACION']
df_estaciones_365

0         1
1         2
2         3
3         4
4         5
       ... 
169     172
170     173
171     174
172     175
217    2008
Name: ESTACION, Length: 173, dtype: int64

In [22]:
# Eliminación de movimientos para estaciones que no tengan mas de 365 dias
# Join con DF de estaciones cargado anteriormente

data_final = pd.merge(data_final, data_estaciones, how='left', left_on='ESTACION', right_on='ESTACION')
data_final

,ANIO,MES,DIA,HORA,ESTACION,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,Es_Festivo,Es_FinSemana,DEMANDA,FECHA,min,max,Diferencia
0,2017,3,31,23,1,6,PM,INVIERNO,1,0,0,7,2017-03-31 23:00:00,2017-03-31 23:00:00,2020-02-29 22:00:00,1064
1,2017,3,31,23,2,6,PM,INVIERNO,1,0,0,3,2017-03-31 23:00:00,2017-03-31 23:00:00,2020-02-29 23:00:00,1065
2,2017,3,31,23,3,6,PM,INVIERNO,1,0,0,1,2017-03-31 23:00:00,2017-03-31 23:00:00,2020-02-29 23:00:00,1065
3,2017,3,31,23,4,6,PM,INVIERNO,1,0,0,1,2017-03-31 23:00:00,2017-03-31 23:00:00,2020-02-29 21:00:00,1064
4,2017,3,31,23,5,6,PM,INVIERNO,1,0,0,2,2017-03-31 23:00:00,2017-03-31 23:00:00,2020-02-29 23:00:00,1065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033210,2020,2,29,23,214,7,PM,INVIERNO,1,0,1,2,2020-02-29 23:00:00,2019-09-11 09:00:00,2020-02-29 23:00:00,171
3033211,2020,2,29,23,215,7,PM,INVIERNO,1,0,1,1,2020-02-29 23:00:00,2019-08-01 03:00:00,2020-02-29 23:00:00,212
3033212,2020,2,29,23,216,7,PM,INVIERNO,1,0,1,1,2020-02-29 23:00:00,2019-09-12 11:00:00,2020-02-29 23:00:00,170
3033213,2020,2,29,23,218,7,PM,INVIERNO,1,0,1,1,2020-02-29 23:00:00,2019-07-10 14:00:00,2020-02-29 23:00:00,234


In [23]:
# Filtro con estaciones con menos de 1 año de datos

data_final = data_final[data_final['Diferencia']>=365]
data_final = data_final[data_final['ESTACION']!=2008]
data_final

,ANIO,MES,DIA,HORA,ESTACION,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,Es_Festivo,Es_FinSemana,DEMANDA,FECHA,min,max,Diferencia
0,2017,3,31,23,1,6,PM,INVIERNO,1,0,0,7,2017-03-31 23:00:00,2017-03-31 23:00:00,2020-02-29 22:00:00,1064
1,2017,3,31,23,2,6,PM,INVIERNO,1,0,0,3,2017-03-31 23:00:00,2017-03-31 23:00:00,2020-02-29 23:00:00,1065
2,2017,3,31,23,3,6,PM,INVIERNO,1,0,0,1,2017-03-31 23:00:00,2017-03-31 23:00:00,2020-02-29 23:00:00,1065
3,2017,3,31,23,4,6,PM,INVIERNO,1,0,0,1,2017-03-31 23:00:00,2017-03-31 23:00:00,2020-02-29 21:00:00,1064
4,2017,3,31,23,5,6,PM,INVIERNO,1,0,0,2,2017-03-31 23:00:00,2017-03-31 23:00:00,2020-02-29 23:00:00,1065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033191,2020,2,29,23,168,7,PM,INVIERNO,1,0,1,2,2020-02-29 23:00:00,2017-03-31 23:00:00,2020-02-29 23:00:00,1065
3033192,2020,2,29,23,169,7,PM,INVIERNO,1,0,1,8,2020-02-29 23:00:00,2017-03-31 23:00:00,2020-02-29 23:00:00,1065
3033193,2020,2,29,23,171,7,PM,INVIERNO,1,0,1,2,2020-02-29 23:00:00,2017-03-31 23:00:00,2020-02-29 23:00:00,1065
3033194,2020,2,29,23,172,7,PM,INVIERNO,1,0,1,1,2020-02-29 23:00:00,2017-04-01 02:00:00,2020-02-29 23:00:00,1064


In [24]:
data_final.groupby('ESTACION').agg(DEMANDA=('DEMANDA', 'count')) # Validación de 179 estaciones

,DEMANDA
ESTACION,
1,19681
2,14344
3,19659
4,16108
5,17600
...,...
171,17302
172,17496
173,10912


In [25]:
#Eliminación de columnas usadas como auxiliares

data_final = data_final.drop(columns=['min', 'max', 'Diferencia'], axis=1)
data_final

,ANIO,MES,DIA,HORA,ESTACION,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,Es_Festivo,Es_FinSemana,DEMANDA,FECHA
0,2017,3,31,23,1,6,PM,INVIERNO,1,0,0,7,2017-03-31 23:00:00
1,2017,3,31,23,2,6,PM,INVIERNO,1,0,0,3,2017-03-31 23:00:00
2,2017,3,31,23,3,6,PM,INVIERNO,1,0,0,1,2017-03-31 23:00:00
3,2017,3,31,23,4,6,PM,INVIERNO,1,0,0,1,2017-03-31 23:00:00
4,2017,3,31,23,5,6,PM,INVIERNO,1,0,0,2,2017-03-31 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033191,2020,2,29,23,168,7,PM,INVIERNO,1,0,1,2,2020-02-29 23:00:00
3033192,2020,2,29,23,169,7,PM,INVIERNO,1,0,1,8,2020-02-29 23:00:00
3033193,2020,2,29,23,171,7,PM,INVIERNO,1,0,1,2,2020-02-29 23:00:00
3033194,2020,2,29,23,172,7,PM,INVIERNO,1,0,1,1,2020-02-29 23:00:00


In [26]:
# Se guarda DF final SIN METEOROLOGIA
data_final.to_csv('../../Data/DataFrame_Final_SinMeteo.csv')

In [27]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2912138 entries, 0 to 3033195
Data columns (total 13 columns):
 #   Column         Dtype         
---  ------         -----         
 0   ANIO           int64         
 1   MES            int64         
 2   DIA            int64         
 3   HORA           int64         
 4   ESTACION       int64         
 5   DIA_SEMANA     int64         
 6   AM_PM          object        
 7   TEMPORADA      object        
 8   TEMPORADA_NUM  int64         
 9   Es_Festivo     int64         
 10  Es_FinSemana   int64         
 11  DEMANDA        int64         
 12  FECHA          datetime64[ns]
dtypes: datetime64[ns](1), int64(10), object(2)
memory usage: 311.0+ MB


### INCORPORACION DE VARIABLES METEOROLOGICAS

In [28]:
db_Meteo = _connect_mongo('cloud', 'cluster0.15npsxw.mongodb.net', None, 'ucmtfm2022', 'UCM_2022', 'BiciMAD', 'Meteo_US_NivelHora')

Conexion OK
Collection(Database(MongoClient(host=['ac-x1d17w2-shard-00-01.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-02.15npsxw.mongodb.net:27017', 'ac-x1d17w2-shard-00-00.15npsxw.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-xyv6ql-shard-0', tls=True, serverselectiontimeoutms=4000, tlscafile='C:\\ProgramData\\Anaconda3\\lib\\site-packages\\certifi\\cacert.pem'), 'BiciMAD'), 'Meteo_US_NivelHora')


In [29]:
# Meteorologia
t_ini = time.time()
data_Meteo = _consulta_meteoUS_full(db_Meteo)
t_end = time.time()

print (t_end - t_ini)
data_Meteo.info()

1.3117105960845947
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49368 entries, 0 to 49367
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ANIO                 49368 non-null  int64  
 1   MES                  49368 non-null  int64  
 2   DIA                  49368 non-null  int64  
 3   HORA                 49368 non-null  int64  
 4   TEMPERATURA          49368 non-null  float64
 5   TEMP_SENS_TERMC      49368 non-null  float64
 6   VIENTO               49368 non-null  float64
 7   PRESION              49368 non-null  float64
 8   HUMEDAD              49368 non-null  float64
 9   PRECIPITACION_1h     49368 non-null  float64
 10  PRECIPITACION_3h     49368 non-null  float64
 11  NIEVE_1h             49368 non-null  float64
 12  NIEVE_3h             49368 non-null  float64
 13  ID_TIPO_TIEMPO       49368 non-null  object 
 14  DESC_TIEMPO          49368 non-null  object 
 15  DESC_TIEMPO_detal

In [30]:
data_Meteo.ANIO.unique()

array([2017, 2018, 2019, 2020, 2021, 2022], dtype=int64)

In [31]:
data_Meteo

,ANIO,MES,DIA,HORA,TEMPERATURA,TEMP_SENS_TERMC,VIENTO,PRESION,HUMEDAD,PRECIPITACION_1h,PRECIPITACION_3h,NIEVE_1h,NIEVE_3h,ID_TIPO_TIEMPO,DESC_TIEMPO,DESC_TIEMPO_detalle
0,2017,1,1,0,-3.42,-5.73,1.54,1030.0,93.0,0.0,0.0,0.0,0.0,800,Clear,sky is clear
1,2017,1,1,1,-4.42,-4.42,1.03,1029.0,93.0,0.0,0.0,0.0,0.0,800,Clear,sky is clear
2,2017,1,1,2,-4.42,-4.42,0.51,1028.0,93.0,0.0,0.0,0.0,0.0,802,Clouds,scattered clouds
3,2017,1,1,3,-3.42,-3.42,1.03,1028.0,86.0,0.0,0.0,0.0,0.0,803,Clouds,broken clouds
4,2017,1,1,4,-2.42,-6.50,3.10,1028.0,86.0,0.0,0.0,0.0,0.0,803,Clouds,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49363,2022,8,19,19,29.39,27.85,1.54,1019.0,23.0,0.0,0.0,0.0,0.0,800,Clear,sky is clear
49364,2022,8,19,20,27.07,26.39,1.03,1006.0,28.0,0.0,0.0,0.0,0.0,800,Clear,sky is clear
49365,2022,8,19,21,26.27,26.27,1.03,1006.0,28.0,0.0,0.0,0.0,0.0,800,Clear,sky is clear
49366,2022,8,19,22,25.11,24.46,3.09,1006.0,30.0,0.0,0.0,0.0,0.0,800,Clear,sky is clear


In [32]:
# #data_final_respaldo = data_final.copy()
# data_final = data_final_respaldo.copy()
# data_final.shape

In [33]:
# MERGE entre datos de tracks y meteorologia

data_final = pd.merge(data_final, data_Meteo, how='left', left_on=['ANIO','MES','DIA','HORA'],
                      right_on=['ANIO','MES','DIA','HORA'])

In [34]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2912138 entries, 0 to 2912137
Data columns (total 25 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ANIO                 int64         
 1   MES                  int64         
 2   DIA                  int64         
 3   HORA                 int64         
 4   ESTACION             int64         
 5   DIA_SEMANA           int64         
 6   AM_PM                object        
 7   TEMPORADA            object        
 8   TEMPORADA_NUM        int64         
 9   Es_Festivo           int64         
 10  Es_FinSemana         int64         
 11  DEMANDA              int64         
 12  FECHA                datetime64[ns]
 13  TEMPERATURA          float64       
 14  TEMP_SENS_TERMC      float64       
 15  VIENTO               float64       
 16  PRESION              float64       
 17  HUMEDAD              float64       
 18  PRECIPITACION_1h     float64       
 19  PRECIPITACION_3h     

In [35]:
# ELIMINACION DE VARIABLES INNECESARIAS
data_final = data_final.drop(columns=['TEMP_SENS_TERMC', 'NIEVE_1h', 'NIEVE_3h','ID_TIPO_TIEMPO'], axis=1)

In [36]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2912138 entries, 0 to 2912137
Data columns (total 21 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ANIO                 int64         
 1   MES                  int64         
 2   DIA                  int64         
 3   HORA                 int64         
 4   ESTACION             int64         
 5   DIA_SEMANA           int64         
 6   AM_PM                object        
 7   TEMPORADA            object        
 8   TEMPORADA_NUM        int64         
 9   Es_Festivo           int64         
 10  Es_FinSemana         int64         
 11  DEMANDA              int64         
 12  FECHA                datetime64[ns]
 13  TEMPERATURA          float64       
 14  VIENTO               float64       
 15  PRESION              float64       
 16  HUMEDAD              float64       
 17  PRECIPITACION_1h     float64       
 18  PRECIPITACION_3h     float64       
 19  DESC_TIEMPO          

In [37]:
"""Chequear los valores registros nulos"""
print("Valores nulos:")
print(data_final.isnull().sum())

Valores nulos:
ANIO                   0
MES                    0
DIA                    0
HORA                   0
ESTACION               0
DIA_SEMANA             0
AM_PM                  0
TEMPORADA              0
TEMPORADA_NUM          0
Es_Festivo             0
Es_FinSemana           0
DEMANDA                0
FECHA                  0
TEMPERATURA            0
VIENTO                 0
PRESION                0
HUMEDAD                0
PRECIPITACION_1h       0
PRECIPITACION_3h       0
DESC_TIEMPO            0
DESC_TIEMPO_detalle    0
dtype: int64


In [38]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2912138 entries, 0 to 2912137
Data columns (total 21 columns):
 #   Column               Dtype         
---  ------               -----         
 0   ANIO                 int64         
 1   MES                  int64         
 2   DIA                  int64         
 3   HORA                 int64         
 4   ESTACION             int64         
 5   DIA_SEMANA           int64         
 6   AM_PM                object        
 7   TEMPORADA            object        
 8   TEMPORADA_NUM        int64         
 9   Es_Festivo           int64         
 10  Es_FinSemana         int64         
 11  DEMANDA              int64         
 12  FECHA                datetime64[ns]
 13  TEMPERATURA          float64       
 14  VIENTO               float64       
 15  PRESION              float64       
 16  HUMEDAD              float64       
 17  PRECIPITACION_1h     float64       
 18  PRECIPITACION_3h     float64       
 19  DESC_TIEMPO          

In [39]:
# REORDENAMIENTO DE COLUMNAS PREVIO A EXPORT

data_final_cierre = data_final[['ESTACION', 'ANIO', 'MES', 'DIA', 'HORA', 'FECHA', 'DIA_SEMANA', 'AM_PM', 'TEMPORADA', 'TEMPORADA_NUM', 
                                'Es_Festivo', 'Es_FinSemana', 'TEMPERATURA', 'VIENTO', 'PRESION', 'HUMEDAD', 'PRECIPITACION_1h', 
                                'PRECIPITACION_3h', 'DESC_TIEMPO', 'DESC_TIEMPO_detalle', 'DEMANDA']]
data_final_cierre

,ESTACION,ANIO,MES,DIA,HORA,FECHA,DIA_SEMANA,AM_PM,TEMPORADA,TEMPORADA_NUM,...,Es_FinSemana,TEMPERATURA,VIENTO,PRESION,HUMEDAD,PRECIPITACION_1h,PRECIPITACION_3h,DESC_TIEMPO,DESC_TIEMPO_detalle,DEMANDA
0,1,2017,3,31,23,2017-03-31 23:00:00,6,PM,INVIERNO,1,...,0,12.58,7.72,1020.0,44.0,0.0,0.0,Clouds,few clouds,7
1,2,2017,3,31,23,2017-03-31 23:00:00,6,PM,INVIERNO,1,...,0,12.58,7.72,1020.0,44.0,0.0,0.0,Clouds,few clouds,3
2,3,2017,3,31,23,2017-03-31 23:00:00,6,PM,INVIERNO,1,...,0,12.58,7.72,1020.0,44.0,0.0,0.0,Clouds,few clouds,1
3,4,2017,3,31,23,2017-03-31 23:00:00,6,PM,INVIERNO,1,...,0,12.58,7.72,1020.0,44.0,0.0,0.0,Clouds,few clouds,1
4,5,2017,3,31,23,2017-03-31 23:00:00,6,PM,INVIERNO,1,...,0,12.58,7.72,1020.0,44.0,0.0,0.0,Clouds,few clouds,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2912133,168,2020,2,29,23,2020-02-29 23:00:00,7,PM,INVIERNO,1,...,1,9.60,6.20,1015.0,69.0,0.0,0.0,Clouds,few clouds,2
2912134,169,2020,2,29,23,2020-02-29 23:00:00,7,PM,INVIERNO,1,...,1,9.60,6.20,1015.0,69.0,0.0,0.0,Clouds,few clouds,8
2912135,171,2020,2,29,23,2020-02-29 23:00:00,7,PM,INVIERNO,1,...,1,9.60,6.20,1015.0,69.0,0.0,0.0,Clouds,few clouds,2
2912136,172,2020,2,29,23,2020-02-29 23:00:00,7,PM,INVIERNO,1,...,1,9.60,6.20,1015.0,69.0,0.0,0.0,Clouds,few clouds,1


In [40]:
# DataFrame Final

data_final_cierre.to_csv('../../Data/DataFrame_Final_Cierre_2017_2019.csv', index=False)